# `withdrawal` data

The withdrawal data is defined at the rally level for each category and identifies the teams/crews that have withdrawn during each stage of the rally.

In [1]:
# Load in the required packages
import pandas as pd
from jupyterlite_simple_cors_proxy import furl, xurl

# Generate the API URL pattern
dakar_api_template = "https://www.dakar.live.worldrallyraidchampionship.com/api/{path}"

# Define the year
YEAR = 2025
# Define the category
CATEGORY = "A"

# Define the API path to the withdrawal resource
# Use a Python f-string to instantiate variable values directly
withdrawal_path = f"withdrawal-{YEAR}-{CATEGORY}"

# Define the URL
withdrawal_url = dakar_api_template.format(path=withdrawal_path)

# Preview the path and the URL
withdrawal_path, withdrawal_url

('withdrawal-2025-A',
 'https://www.dakar.live.worldrallyraidchampionship.com/api/withdrawal-2025-A')

Within the withdrawal data, we get a list of withdrawals by stage.

In [2]:
# Load in data
# Use furl() to handle CORS issues in Jupyterlite
withdrawal_df = pd.read_json(furl(withdrawal_url))

# Use the stage number as the index
withdrawal_df.set_index("stage", drop=False, inplace=True)
withdrawal_df

,_id,_bind,_updatedAt,_parent,list,stage
stage,,,,,,
4,withdrawal-2025-A-4,withdrawal-2025-A,1736787229800,stage-2025-A:8002af597f2d091dc91450266fab52d6,"[{'bib': 219, 'reason': '2', 'team': {'bib': 2...",4
8,withdrawal-2025-A-8,withdrawal-2025-A,1736839908487,stage-2025-A:d7ab3acebdb4cd17e8d4af9f122fc425,"[{'bib': 619, 'reason': '1', 'team': {'bib': 6...",8
7,withdrawal-2025-A-7,withdrawal-2025-A,1736787226508,stage-2025-A:fadf454f8df8f758e326ad268007a286,"[{'bib': 202, 'reason': '1', 'team': {'bib': 2...",7
5,withdrawal-2025-A-5,withdrawal-2025-A,1736787227298,stage-2025-A:e2b95b95a4ae9063eb4924fc2d7cf7e7,"[{'bib': 208, 'reason': '1', 'team': {'bib': 2...",5
9,withdrawal-2025-A-9,withdrawal-2025-A,1736838862458,stage-2025-A:5f83172bf9f0bb97d14b83a7c0099fa8,"[{'bib': 320, 'reason': '1', 'team': {'bib': 3...",9
3,withdrawal-2025-A-3,withdrawal-2025-A,1736787232655,stage-2025-A:127ed31c3eff071ba6a7e88fe01083ef,"[{'bib': 225, 'reason': '1', 'team': {'bib': 2...",3
2,withdrawal-2025-A-2,withdrawal-2025-A,1736787228730,stage-2025-A:9afdc50df69247c38b23cef4ed50bd14,"[{'bib': 223, 'reason': '1', 'team': {'bib': 2...",2
6,withdrawal-2025-A-6,withdrawal-2025-A,1736787231623,stage-2025-A:33ee273dbd14a1657513f7dab64a635d,"[{'bib': 239, 'reason': '1', 'team': {'bib': 2...",6


In [3]:
withdrawals_by_stage = withdrawal_df["list"].explode()
withdrawals_by_stage_index = withdrawals_by_stage.index

withdrawals_by_stage_df = pd.json_normalize(withdrawals_by_stage)

withdrawals_by_stage_df["stage"] = withdrawals_by_stage_index
withdrawals_by_stage_df.head()

,bib,reason,team.bib,team.brand,team.model,team.vehicle,team.vehicleImg,team.clazz,team.w2rc,team.competitors,stage
0,219,2,219,DACIA,SANDRIDER,THE DACIA SANDRIDERS,https://img.aso.fr/core_app/img-motorSports-da...,96c0869600e0013dbf5f86f60e5c4da4,True,"[{'name': 'S. LOEB', 'firstName': 'SEBASTIEN',...",4
1,236,1,236,MD,OPTIMUS,MD RALLYE SPORT,https://img.aso.fr/core_app/img-motorSports-da...,f00d7ec8d2d96e9cf11aa515109376cf,False,"[{'name': 'E. AMOS', 'firstName': 'EUGENIO', '...",4
2,258,1,258,RED-LINED,NAVARRA VK 56,WALCHER RACING TEAM,https://img.aso.fr/core_app/img-motorSports-da...,f666973e89db183ecfefc75c3af8ffb1,False,"[{'name': 'M. WALCHER', 'firstName': 'MARKUS',...",4
3,337,1,337,BRP,CAN-AM MAVERICK X3,COLORADO MOTORSPORT,NaN,a0a6386a4b9a61b73b036a50966345c0,False,"[{'name': 'C. LUMSDEN', 'firstName': 'CRAIG', ...",4
4,408,1,408,BRP,CAN-AM MAVERICK R,SOUTH RACING CAN-AM,https://img.aso.fr/core_app/img-motorSports-da...,9a68ed3c41c5c7a1642df5d93458baa6,True,"[{'name': 'D. MARTINEZ', 'firstName': 'DIEGO',...",4


The withdrawal data also includes a `reason` column, although I haven't (yet!) found a metadata feed that explains the `reason` code values.

Crew (that is, `competitors`) data is provided as a list of details for each crew member. We can extract this data into its own table, and widen it by adding the `stage` and `reason` data, to provide an alternative table for looking up withdrawal information by competitor name.

In [4]:
withdrawn_competitors_df = (
        withdrawals_by_stage_df[['stage', 'reason', 'bib', 'team.competitors']]
        .explode('team.competitors')
        .reset_index(drop=True)
    )

# Normalize the dictionary contents and combine with team_bib
withdrawn_competitors_df = pd.concat([
    withdrawn_competitors_df[['stage', 'bib', 'reason']],
    pd.json_normalize(withdrawn_competitors_df['team.competitors'])
    ], axis=1)

withdrawn_competitors_df.head()

,stage,bib,reason,name,firstName,lastName,role,gender,nationality,profil,profil_sm,podium,aid
0,4,219,2,S. LOEB,SEBASTIEN,LOEB,P,m,fra,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,5dd0184f-5f90-4b42-b675-88d6501b9965
1,4,219,2,F. LURQUIN,FABIAN,LURQUIN,C,m,bel,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,c640056a-79d0-4454-8e6a-c8b303620020
2,4,236,1,E. AMOS,EUGENIO,AMOS,P,m,ita,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,0575de36-4a38-431c-8892-ac9514c8ccd9
3,4,236,1,P. CECI,PAOLO,CECI,C,m,ita,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,fdacd829-75f9-4cfa-9992-636c8244481e
4,4,258,1,M. WALCHER,MARKUS,WALCHER,P,m,deu,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,93be84ca-0b2b-4da0-933a-e394a8a29cdb


We note, however, that much of this information is generic and could exist in a separate *competitors* data table, with just a unique key reference value, such as the `name`, linking the withdrawn competitors table to the *competitors* data table.

A similar approach could be applied to the `team` values, extracting the data to a separate *teams* data table and using the `team.bib` as the unique key.

In [5]:
team_cols = [c for c in withdrawals_by_stage_df.columns if c.startswith("team")]

# Create a new dataframe, rather than a reference, by using .copy()
withdrawn_teams_df = withdrawals_by_stage_df[team_cols].copy()
withdrawn_teams_df.drop("team.competitors", axis=1, inplace=True)

withdrawn_teams_df.head()

,team.bib,team.brand,team.model,team.vehicle,team.vehicleImg,team.clazz,team.w2rc
0,219,DACIA,SANDRIDER,THE DACIA SANDRIDERS,https://img.aso.fr/core_app/img-motorSports-da...,96c0869600e0013dbf5f86f60e5c4da4,True
1,236,MD,OPTIMUS,MD RALLYE SPORT,https://img.aso.fr/core_app/img-motorSports-da...,f00d7ec8d2d96e9cf11aa515109376cf,False
2,258,RED-LINED,NAVARRA VK 56,WALCHER RACING TEAM,https://img.aso.fr/core_app/img-motorSports-da...,f666973e89db183ecfefc75c3af8ffb1,False
3,337,BRP,CAN-AM MAVERICK X3,COLORADO MOTORSPORT,NaN,a0a6386a4b9a61b73b036a50966345c0,False
4,408,BRP,CAN-AM MAVERICK R,SOUTH RACING CAN-AM,https://img.aso.fr/core_app/img-motorSports-da...,9a68ed3c41c5c7a1642df5d93458baa6,True


Create a simpler withdrawals table:

In [6]:
withdrawals_df = withdrawn_competitors_df[["stage", "bib", "reason", "name"]].copy()
withdrawals_df.head()

,stage,bib,reason,name
0,4,219,2,S. LOEB
1,4,219,2,F. LURQUIN
2,4,236,1,E. AMOS
3,4,236,1,P. CECI
4,4,258,1,M. WALCHER


And tidy up the competitors table to just leave competitor (and team `bib`) data:

In [7]:
withdrawn_competitors_df.drop("stage", axis=1, inplace=True)
withdrawn_competitors_df.drop("reason", axis=1, inplace=True)

withdrawn_competitors_df.head()

,bib,name,firstName,lastName,role,gender,nationality,profil,profil_sm,podium,aid
0,219,S. LOEB,SEBASTIEN,LOEB,P,m,fra,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,5dd0184f-5f90-4b42-b675-88d6501b9965
1,219,F. LURQUIN,FABIAN,LURQUIN,C,m,bel,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,c640056a-79d0-4454-8e6a-c8b303620020
2,236,E. AMOS,EUGENIO,AMOS,P,m,ita,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,0575de36-4a38-431c-8892-ac9514c8ccd9
3,236,P. CECI,PAOLO,CECI,C,m,ita,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,fdacd829-75f9-4cfa-9992-636c8244481e
4,258,M. WALCHER,MARKUS,WALCHER,P,m,deu,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,https://img.aso.fr/core_app/img-motorSports-da...,93be84ca-0b2b-4da0-933a-e394a8a29cdb
